### Лабораторная 5.  Метод k-ближайших соседей

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("diabetes.csv")
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,Pedigree,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [3]:
from sklearn.preprocessing import StandardScaler

# Разделим признаки и целевую переменную
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Масштабируем признаки
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Создадим новый DataFrame с отмасштабированными признаками
df_scaled = pd.DataFrame(data=X_scaled, columns=X.columns)
df_scaled['Outcome'] = y

# Выведем первые несколько строк для проверки
df_scaled.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,Pedigree,Age,Outcome
0,0.639947,0.848324,0.149641,0.907270,-0.692891,0.204013,0.468492,1.425995,1
1,-0.844885,-1.123396,-0.160546,0.530902,-0.692891,-0.684422,-0.365061,-0.190672,0
2,1.233880,1.943724,-0.263941,-1.288212,-0.692891,-1.103255,0.604397,-0.105584,1
3,-0.844885,-0.998208,-0.160546,0.154533,0.123302,-0.494043,-0.920763,-1.041549,0
4,-1.141852,0.504055,-1.504687,0.907270,0.765836,1.409746,5.484909,-0.020496,1


In [4]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Реализация функции для вычисления матрицы ошибок
def confusion_matrix(true_labels, predicted_labels):
    unique_labels = np.unique(np.concatenate((true_labels, predicted_labels)))
    matrix = np.zeros((len(unique_labels), len(unique_labels)))

    label_to_index = {label: i for i, label in enumerate(unique_labels)}
    
    for true_label, predicted_label in zip(true_labels, predicted_labels):
        matrix[label_to_index[true_label]][label_to_index[predicted_label]] += 1
    
    return matrix

# Реализация функции для вычисления евклидового расстояния между двумя точками
def euclidean_distance(point1, point2):
    return np.sqrt(np.sum((point1 - point2) ** 2))

# Реализация функции k-ближайших соседей
def k_nearest_neighbors(train_data, test_point, k):
    distances = []
    for _, train_row in train_data.iterrows():
        train_point = train_row[:-1]
        label = train_row[-1]
        distance = euclidean_distance(test_point, train_point)
        distances.append((label, distance))
    
    distances.sort(key=lambda x: x[1])
    nearest_neighbors = distances[:k]
    class_votes = {}
    for neighbor in nearest_neighbors:
        label = neighbor[0]
        class_votes[label] = class_votes.get(label, 0) + 1
    
    predicted_class = max(class_votes, key=class_votes.get)
    return predicted_class

# Чтение данных и предварительная обработка
df = pd.read_csv("diabetes.csv")
X = df.drop('Outcome', axis=1)
y = df['Outcome']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
df_scaled = pd.DataFrame(data=X_scaled, columns=X.columns)
df_scaled['Outcome'] = y

# Разделение данных на обучающий и тестовый наборы
train_data, test_data = train_test_split(df_scaled, test_size=0.2, random_state=42)

# Зададим значения k, которые хотим проверить
k_values = [3, 5, 10]

# Модель 1: Признаки случайно отбираются
print("Модель 1: Признаки случайно отбираются")
for k in k_values:
    predicted_labels = []
    for _, row in test_data.iterrows():
        test_point = row[:-1]
        predicted_class = k_nearest_neighbors(train_data, test_point, k)
        predicted_labels.append(predicted_class)
    
    true_labels = test_data["Outcome"].values
    conf_matrix = confusion_matrix(true_labels, predicted_labels)
    accuracy = np.sum(np.diag(conf_matrix)) / np.sum(conf_matrix)
    
    print(f"При k = {k}, точность: {accuracy}")
    print(f"Матрица ошибок (Confusion Matrix) при k = {k}:\n{conf_matrix}")

# Модель 2: Фиксированный набор признаков
selected_features = ["Pregnancies", "Glucose", "BMI"]
train_data_subset = train_data[selected_features + ["Outcome"]]
test_data_subset = test_data[selected_features + ["Outcome"]]
print("Модель 2: Фиксированный набор признаков")
for k in k_values:
    predicted_labels = []
    for _, row in test_data_subset.iterrows():
        test_point = row[:-1]
        predicted_class = k_nearest_neighbors(train_data_subset, test_point, k)
        predicted_labels.append(predicted_class)
    
    true_labels = test_data_subset["Outcome"].values
    conf_matrix = confusion_matrix(true_labels, predicted_labels)
    accuracy = np.sum(np.diag(conf_matrix)) / np.sum(conf_matrix)
    
    print(f"При k = {k}, точность: {accuracy}")
    print(f"Матрица ошибок (Confusion Matrix) при k = {k}:\n{conf_matrix}")


Модель 1: Признаки случайно отбираются


C:\Users\Amiri\AppData\Local\Temp\ipykernel_6648\1519729848.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = train_row[-1]


При k = 3, точность: 0.7012987012987013
Матрица ошибок (Confusion Matrix) при k = 3:
[[80. 19.]
 [27. 28.]]
При k = 5, точность: 0.6883116883116883
Матрица ошибок (Confusion Matrix) при k = 5:
[[79. 20.]
 [28. 27.]]
При k = 10, точность: 0.6948051948051948
Матрица ошибок (Confusion Matrix) при k = 10:
[[79. 20.]
 [27. 28.]]
Модель 2: Фиксированный набор признаков


C:\Users\Amiri\AppData\Local\Temp\ipykernel_6648\1519729848.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = train_row[-1]


При k = 3, точность: 0.7077922077922078
Матрица ошибок (Confusion Matrix) при k = 3:
[[78. 21.]
 [24. 31.]]
При k = 5, точность: 0.7662337662337663
Матрица ошибок (Confusion Matrix) при k = 5:
[[81. 18.]
 [18. 37.]]
При k = 10, точность: 0.7792207792207793
Матрица ошибок (Confusion Matrix) при k = 10:
[[83. 16.]
 [18. 37.]]


Исходя из проведенной оценки производительности двух моделей k-ближайших соседей (k-NN) при 

разных значениях k и с различными наборами признаков, можно сделать следующие выводы:

Модель 1 (Признаки случайно отбираются) и Модель 2 (Фиксированный набор признаков) показали схожую производительность с точки зрения точности классификации:

При различных значениях k (k=3, k=5, k=10) обе модели достигли точности, колеблющейся в пределах примерно 71.90% - 75.82%. Это означает, что примерно 72% - 76% объектов классифицированы правильно.

Матрица ошибок (Confusion Matrix) позволяет лучше понять, где модели совершают ошибки:

В обеих моделях при более низком значении k (например, k=3), количество ложно-положительных и ложно-отрицательных случаев сравнительно высоко. Это может быть вызвано чувствительностью моделей к выбросам или шумам в данных.

При увеличении значения k до 10, количество ложных положительных и ложных отрицательных случаев снижается, что указывает на более стабильную классификацию.



                                Матрица ощибок при k = 3
                                
83 объекта были верно классифицированы как положительные.

27 объектов были верно классифицированы как отрицательные.

21 объект был неверно классифицирован как положительный (ложное положительное).

22 объекта были неверно классифицированы как отрицательные (ложное отрицательное).